In [3]:
base_dir="/mnt/efs/shared/meg_shared_scripts/meg-kb"
data_ac="indeeda-meg-ac"
data_pt="indeeda-meg-pt"
yutong_base_dir="/home/ubuntu/users/yutong"

In [4]:
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


In [5]:
from tqdm.notebook import tqdm
import argparse
import re
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, entropy, gmean
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import json
from collections import defaultdict
import time
import importlib

import logging
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import os
import sys
import math
from annoy import AnnoyIndex
import matplotlib
from matplotlib import pyplot as plt
import networkx as nx

import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
spacy_tokenizer = nlp.tokenizer


from compute_concept_clusters import knn
from compute_keyphrase_embeddings import ensure_tensor_on_device, mean_pooling

from lm_probes import LMProbe, LMProbe_GPT2, LMProbe_Joint, LMProbe_PMI, LMProbe_PMI_greedy
from utils import load_embeddings, load_seed_aligned_concepts, load_seed_aligned_relations, load_benchmark
from utils import get_masked_contexts, bert_untokenize
from utils import learn_patterns

from roberta_ses.interface import Roberta_SES_Entailment

In [6]:
import utils
importlib.reload(utils)
from utils import load_embeddings, load_seed_aligned_concepts, load_seed_aligned_relations, load_benchmark
from utils import get_masked_contexts, bert_untokenize
from utils import learn_patterns

In [7]:
seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
seed_aligned_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_relations_nodup.csv')
benchmark_path = os.path.join(base_dir, f'data/indeed-benchmark/benchmark_evidence_clean.csv')


# EE-LM-probe (prompt)

In [ ]:
'''
@Nikita: Here are the code blocks for exploring LM prompts for EE.

Some core code:
lm_probe = LMProbe()        // LMProbe: BERT; LMProbe_GPT2: GPT2; etc.
all_entitites = ...         // all the entities
_template = "Dress code like jeans, [MASK] and tattoos." // A string with [MASK] (LMProbe automatically takes care of all entity token lengths, so don't need to duplicate mask tokens)
_res = lm_probe.score_candidates(input_txt=_template, cands=all_entities)
list(enumerate(_res[:50]))  // Show results (rank, cand, score)

'''

In [9]:
lm_probe = LMProbe()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
d_lm_probe = LMProbe('/home/ubuntu/users/nikita/models/bert_finetuned_lm/indeed_reviews_ques_ans')

In [11]:
seed_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
seed_concepts_df = load_seed_aligned_concepts(seed_concepts_path)
print(seed_concepts_df.head())
emb_num_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembednum+seeds.txt')
with open(emb_num_path, 'r') as f:
    all_entities = [l.rsplit(' ', 1)[0] for l in f]
print(all_entities[:20])
all_entities = list(set(all_entities))

seed_instances_dict = dict(zip(
        seed_concepts_df['alignedCategoryName'].tolist(),
        seed_concepts_df['seedInstances'].tolist()
    ))
    
seed_entities_lst = seed_concepts_df['seedInstances'].tolist()
seed_entities = [item for sublist in seed_entities_lst for item in sublist]
print(seed_entities)
concepts = seed_concepts_df['alignedCategoryName'].tolist()

  alignedCategoryName unalignedCategoryName generalizations  \
0             company               company             NaN   
1          dress_code            dress code             NaN   
2        job_position          job position             NaN   
3        pay_schedule            pay period             NaN   
4            benefits              benefits    compensation   

                                       seedInstances  
0       [walmart, amazon, subway, microsoft, target]  
1  [business casual, uniform, hair color, tattoos...  
2  [delivery driver, store manager, cashier, pack...  
3               [weekly, biweekly, friday, saturday]  
4  [health insurance, flexible schedule, 401k, pa...  
['multiple times', 'upper', 'management', 'wal mart', 'company', 'overnight stocker', 'walmart', 'prejudice', 'tuition assistance', 'mgr', 'department', 'manager', 'knowledge', 'leadership', 'job security', 'current position', 'positive attitude', 'business', '* *', 'talk']
['walmart', 'ama

In [ ]:
gold_ee = pd.read_csv('../../../tmp/gold_ee.csv')
gold_ee = gold_ee[~gold_ee['neighbor'].isin(seed_entities)]
gold_ee = gold_ee.drop_duplicates(subset=['concept', 'neighbor'])

from tabulate import tabulate

def pratk(df, score='lm_score'):
    df = pd.merge(df, gold_ee, on=['concept', 'neighbor'])
    df = df.fillna(1)
    grped = df.groupby('concept')
    scores = {}
    for name, grp in grped:
        grp = grp.reset_index()
        grp = grp.sort_values(by=score, ascending=False)
        ks = [1,2,5,10,25,50,75,100,200]
        ks = [k for k in ks if k < len(grp)]
        all_correct = len(grp[grp['Majority'] > 0])
        patk = {}
        ratk = {}
        mink = {}
        for k in ks:
            sub = grp.head(k)
            correct = sub[sub['Majority'] > 0]
            precision = sub['Majority'].sum() / k
            recall = sub['Majority'].sum() / all_correct
            patk[k] = precision
            ratk[k] = recall
            mink[k] = correct.iloc[-1][score] if len(correct) > 0 else -1
        scores[name] = {"precision": patk, "recall": ratk, "min_score": mink}
    for cc, c_scores in scores.items():
        print(cc)
        print(pd.DataFrame(c_scores).to_string())
    return scores

## Avg. Scoring

In [13]:
def probe_and_eval(lm_probe, concept, probe_prompts, concept_phrase=None, evaluate=True):
    seeds = seed_instances_dict[concept]
    if concept_phrase is None:
        cc_phrase = ' '.join(concept.split('_'))
    else:
        cc_phrase = concept_phrase
    cand_scores_per_template = []
    cand_scores = []
    for template in probe_prompts:
        _input_txt = template.format(cc_phrase, ', '.join(seeds[:-1]), '[MASK]', seeds[-1])
        _cand_scores = lm_probe.score_candidates(_input_txt, all_entities)
        _cand_scores.sort(key=lambda d : d["cand"])
        cand_scores_per_template.append(_cand_scores)

    for _cand_score_lst in zip(*cand_scores_per_template):
                    # _cand_score_lst: List[Dict["cand", "score"]], for the same "cand" and different template 
            _cand = _cand_score_lst[0]["cand"]
            _score = sum([d["score"] for d in _cand_score_lst]) / len(_cand_score_lst)
                    # _score = np.log(_score)
            cand_scores.append({"cand": _cand, "score": _score})

    extraction_results = []
    for d in cand_scores:
        e = d["cand"]
        if e in seeds:
            continue
        lm_score = d["score"]
        extraction_results.append({'concept': concept,
                                           'neighbor': e,
                                           'lm_score': lm_score
                                          })

        extraction_results.sort(key=lambda d : d['lm_score'], reverse=True)
        results_df = pd.DataFrame(extraction_results)
    if evaluate:
        pratk(results_df)
    return results_df

In [14]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "{2} has many employees.",
        "I worked at {2}",
        "hired at {2}."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'company', probe_prompts)
print('with relation prompts')
probe_and_eval(lm_probe,'company', probe_prompts2)

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'company', probe_prompts)
print('with relation prompts')
probe_and_eval(d_lm_probe,'company', probe_prompts2)

BERT-base
company
     precision    recall  min_score
1     1.000000  0.008772   0.068087
2     1.000000  0.017544   0.046398
5     1.000000  0.043860   0.020752
10    1.000000  0.087719   0.010284
25    1.000000  0.219298   0.002511
50    0.840000  0.368421   0.000986
75    0.706667  0.464912   0.000690
100   0.600000  0.526316   0.000380
with relation prompts
company
     precision    recall  min_score
1     1.000000  0.008772   0.034074
2     1.000000  0.017544   0.023606
5     1.000000  0.043860   0.012628
10    1.000000  0.087719   0.005199
25    0.960000  0.210526   0.001541
50    0.780000  0.342105   0.000706
75    0.706667  0.464912   0.000488
100   0.600000  0.526316   0.000320

Domain-adapted BERT
company
     precision    recall  min_score
1         1.00  0.008772   0.063044
2         1.00  0.017544   0.034719
5         1.00  0.043860   0.014492
10        1.00  0.087719   0.009953
25        1.00  0.219298   0.005252
50        0.88  0.385965   0.001830
75        0.76  0.50000

,concept,neighbor,lm_score
0,company,starbucks,3.476803e-02
1,company,mcdonalds,1.984275e-02
2,company,nike,1.751212e-02
3,company,amazons,1.739639e-02
4,company,mcdonald ' s,1.101652e-02
...,...,...,...
8032,company,gdansk,3.600702e-08
8033,company,screenplay,3.215033e-08
8034,company,mascara,2.928107e-08
8035,company,flute,2.463806e-08


In [17]:
_results_df = probe_and_eval(d_lm_probe, 'company', probe_prompts)

company
     precision    recall  min_score
1         1.00  0.008772   0.063044
2         1.00  0.017544   0.034719
5         1.00  0.043860   0.014492
10        1.00  0.087719   0.009953
25        1.00  0.219298   0.005252
50        0.88  0.385965   0.001830
75        0.76  0.500000   0.000813
100       0.72  0.631579   0.000405


In [22]:
''' YS: "fasts" and "etcs" are not company, why P@10 = 1.00 ?? '''

_results_df.head(10)

,concept,neighbor,lm_score
0,company,starbucks,0.063044
1,company,nike,0.034719
2,company,amazons,0.025174
3,company,apple,0.018960
4,company,etcs,0.018090
5,company,macs,0.017288
6,company,fasts,0.014690
7,company,mcdonalds,0.014492
8,company,wal mart,0.012479
9,company,wal mart,0.012479


In [102]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "The company did not do {2}.",
        "They check your {2}.",
        "{2} is not a card."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'background_screening', probe_prompts)
print('with relation prompts')
probe_and_eval(lm_probe, 'background_screening', probe_prompts2)

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'background_screening', probe_prompts)
print('with relation prompts')
probe_and_eval(d_lm_probe,'background_screening', probe_prompts2)

BERT-base
background_screening
     precision    recall  min_score
1     1.000000  0.012346   0.068725
2     1.000000  0.024691   0.061470
5     0.800000  0.049383   0.027790
10    0.800000  0.098765   0.010895
25    0.720000  0.222222   0.006133
50    0.680000  0.419753   0.004163
75    0.546667  0.506173   0.002976
100   0.580000  0.716049   0.002050
with relation prompts
background_screening
     precision    recall  min_score
1         1.00  0.012346   0.034897
2         1.00  0.024691   0.030772
5         0.80  0.049383   0.013992
10        0.90  0.111111   0.006143
25        0.76  0.234568   0.003983
50        0.68  0.419753   0.002365
75        0.60  0.555556   0.001736
100       0.58  0.716049   0.001147

Domain-adapted BERT
background_screening
     precision    recall  min_score
1     0.000000  0.000000  -1.000000
2     0.000000  0.000000  -1.000000
5     0.600000  0.037037   0.085545
10    0.700000  0.086420   0.024580
25    0.720000  0.222222   0.009247
50    0.600000  0.37

In [103]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "The job requires {2}.",
        "You need to have {2} to apply.",
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'hire_prerequisite', probe_prompts, 'requirements')
print('with relation prompts')
probe_and_eval(lm_probe, 'hire_prerequisite', probe_prompts2, 'requirements')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'hire_prerequisite', probe_prompts, 'requirements')
print('with relation prompts')
probe_and_eval(d_lm_probe,'hire_prerequisite', probe_prompts2, 'requirements')

BERT-base
hire_prerequisite
     precision  recall  min_score
1     1.000000    0.01   0.052757
2     1.000000    0.02   0.013271
5     0.400000    0.02   0.013271
10    0.500000    0.05   0.005917
25    0.560000    0.14   0.004174
50    0.660000    0.33   0.002382
75    0.626667    0.47   0.001623
100   0.610000    0.61   0.001089
200   0.495000    0.99   0.000005
with relation prompts
hire_prerequisite
     precision  recall  min_score
1     1.000000    0.01   0.031668
2     0.500000    0.01   0.031668
5     0.600000    0.03   0.008726
10    0.500000    0.05   0.004849
25    0.600000    0.15   0.003153
50    0.640000    0.32   0.001702
75    0.653333    0.49   0.001265
100   0.620000    0.62   0.000885
200   0.495000    0.99   0.000004

Domain-adapted BERT
hire_prerequisite
     precision  recall  min_score
1     1.000000    0.01   0.019661
2     1.000000    0.02   0.016057
5     0.800000    0.04   0.005686
10    0.800000    0.08   0.003724
25    0.880000    0.22   0.002821
50    0.7

In [104]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "they will hire people {2}.",
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'person', probe_prompts, 'person')
print('with relation prompts')
probe_and_eval(lm_probe, 'person', probe_prompts2, 'person')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'person', probe_prompts, 'person')
print('with relation prompts')
probe_and_eval(d_lm_probe,'person', probe_prompts2, 'person')

BERT-base
person
     precision    recall  min_score
1     1.000000  0.029412   0.027419
2     1.000000  0.058824   0.015791
5     0.800000  0.117647   0.005052
10    0.900000  0.264706   0.002278
25    0.640000  0.470588   0.001025
50    0.440000  0.647059   0.000608
75    0.346667  0.764706   0.000365
100   0.280000  0.823529   0.000266
with relation prompts
person
     precision    recall  min_score
1     1.000000  0.029412   0.020585
2     1.000000  0.058824   0.011879
5     0.800000  0.117647   0.003799
10    0.900000  0.264706   0.001718
25    0.680000  0.500000   0.000803
50    0.460000  0.676471   0.000445
75    0.346667  0.764706   0.000278
100   0.280000  0.823529   0.000204

Domain-adapted BERT
person
     precision    recall  min_score
1         1.00  0.029412   0.036885
2         1.00  0.058824   0.019011
5         1.00  0.147059   0.012808
10        0.60  0.176471   0.011755
25        0.68  0.500000   0.002846
50        0.48  0.705882   0.001528
75        0.36  0.794118  

In [106]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "they allow you to wear {2}.",
        "{2} are allowed."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'dress_code', probe_prompts, 'dress_code')
print('with relation prompts')
probe_and_eval(lm_probe, 'dress_code', probe_prompts2, 'dress_code')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'dress_code', probe_prompts, 'dress_code')
print('with relation prompts')
probe_and_eval(d_lm_probe,'dress_code', probe_prompts2, 'dress_code')

BERT-base
dress_code
     precision    recall  min_score
1         1.00  0.003922   0.044785
2         1.00  0.007843   0.030868
5         1.00  0.019608   0.017537
10        1.00  0.039216   0.008212
25        0.96  0.094118   0.002613
50        0.92  0.180392   0.001255
75        0.84  0.247059   0.000754
100       0.86  0.337255   0.000488
200       0.89  0.698039   0.000045
with relation prompts
dress_code
     precision    recall  min_score
1     1.000000  0.003922   0.026927
2     1.000000  0.007843   0.018830
5     1.000000  0.019608   0.010764
10    1.000000  0.039216   0.006065
25    0.960000  0.094118   0.002116
50    0.980000  0.192157   0.001280
75    0.933333  0.274510   0.000853
100   0.880000  0.345098   0.000605
200   0.885000  0.694118   0.000108

Domain-adapted BERT
dress_code
     precision    recall  min_score
1         0.00  0.000000  -1.000000
2         0.50  0.003922   0.056419
5         0.80  0.015686   0.033686
10        0.90  0.035294   0.015546
25        0.88

In [72]:
corpus_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/sent_segmentation.txt')
with open(corpus_path, 'r') as f:
    corpus = f.readlines()
    corpus = [l.strip() for l in corpus]
print(len(corpus))
corpus[:20]

901796


["Hard , unless you 're favorited .",
 'I asked <phrase>multiple times</phrase> to be trained on other things through out the store and refused .',
 'nearly impossible , unless you have an in with <phrase>upper</phrase> level <phrase>management</phrase> your better off finding a better job anywhere else',
 'Very easy to get promoted as long as you work hard show your motivated and are reliable .',
 'It can be really easy to get promoted !',
 'The <phrase>management</phrase> team , promotes associates to <phrase>management</phrase> and above within the store , before they look for someone outside <phrase>wal mart</phrase>',
 'It is difficult to be promoted .',
 'A lot of factors go into a store employee promotion .',
 "It 's pretty easy to get the job done as long as you are on time , respectful , kind , trustworthy and get the job done when it 's suppose to .",
 "you 'll be promoted .",
 'Easy if you are willing to work around more hours and little pay raise .',
 'not hard at all',
 'V

In [78]:
import collections
def get_context(entities, corpus):
    
    matched_context = []
    contexts = []
    for entity in entities: 
        token = '<phrase>{}</phrase>'.format(entity)
        for sent in corpus:
            if token in sent:
                matched_context.append(sent)
        for sent in matched_context:
            splits = sent.split(token)
            if len(splits) < 2:
                continue
            left_context_words = splits[0].split(' ')
            left_context_words = left_context_words[-min(len(left_context_words), 4):]

            right_context_words = splits[1].split(' ')
            right_context_words = right_context_words[:min(len(right_context_words), 4)]
            context = ' '.join(left_context_words) + '<tgt>' + ' '.join(right_context_words)
            context_words = context.split(' ')
            if len(context_words) < 4:
                continue
            contexts.append(context)
    print(collections.Counter(contexts).most_common(20))

[('to work at <tgt> .', 63), ('<tgt> does not drug', 56), ('to work at <tgt>', 50), ('to work for <tgt>', 22), ('No <tgt> does not drug', 19), ('while working at <tgt> .', 17), ('<tgt> is a good', 15), ('<tgt> is a great', 15), ('No , <tgt> does not drug', 15), ('<tgt> does not hire', 14), ('to work for <tgt> .', 13), ('<tgt> is a <phrase>great', 13), ('<tgt> does not do', 12), ('No <tgt> does not hire', 11), ('<tgt> will work with', 10), ('<tgt> is always hiring', 10), ('The <tgt> I worked at', 10), ('<tgt> pays bi weekly', 10), ('from home for <tgt>', 10), ('<tgt> is a <phrase>drug', 10)]


In [81]:
get_context(['walmart', 'amazon', 'subway', 'microsoft', 'target'], corpus)

[('to work at <tgt> .', 63), ('<tgt> does not drug', 56), ('to work at <tgt>', 50), ('to work for <tgt>', 22), ('No <tgt> does not drug', 19), ('while working at <tgt> .', 17), ('<tgt> is a good', 15), ('<tgt> is a great', 15), ('No , <tgt> does not drug', 15), ('<tgt> does not hire', 14), ('to work for <tgt> .', 13), ('<tgt> is a <phrase>great', 13), ('<tgt> does not do', 12), ('No <tgt> does not hire', 11), ('<tgt> will work with', 10), ('<tgt> is always hiring', 10), ('The <tgt> I worked at', 10), ('<tgt> pays bi weekly', 10), ('from home for <tgt>', 10), ('<tgt> is a <phrase>drug', 10)]


In [108]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]

probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "to work at {2}.",
        "{2} does not drug.",
        "while working at {2}."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'company', probe_prompts, 'company')
print('with relation prompts')
probe_and_eval(lm_probe, 'company', probe_prompts2, 'company')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'company', probe_prompts, 'company')
print('with relation prompts')
probe_and_eval(d_lm_probe,'company', probe_prompts2, 'company')

BERT-base
company
     precision    recall  min_score
1     1.000000  0.008772   0.068087
2     1.000000  0.017544   0.046398
5     1.000000  0.043860   0.020752
10    1.000000  0.087719   0.010284
25    1.000000  0.219298   0.002511
50    0.840000  0.368421   0.000986
75    0.706667  0.464912   0.000690
100   0.600000  0.526316   0.000380
with relation prompts
company
     precision    recall  min_score
1     1.000000  0.008772   0.034133
2     1.000000  0.017544   0.023354
5     1.000000  0.043860   0.012258
10    1.000000  0.087719   0.005157
25    0.960000  0.210526   0.001551
50    0.780000  0.342105   0.000703
75    0.666667  0.438596   0.000495
100   0.590000  0.517544   0.000362

Domain-adapted BERT
company
     precision    recall  min_score
1         1.00  0.008772   0.063044
2         1.00  0.017544   0.034719
5         1.00  0.043860   0.014492
10        1.00  0.087719   0.009953
25        1.00  0.219298   0.005252
50        0.88  0.385965   0.001830
75        0.76  0.50000

In [83]:
get_context(['drug test', 'criminal background check', 'employment verification', 'driving record', 'credit report', 'criminal record'], corpus)

[('to take a <tgt> .', 167), ('to take a <tgt>', 114), ('Yes they do <tgt>', 85), ('Yes they do <tgt> .', 44), ('to do a <tgt> .', 40), ('Yes they <tgt> .', 38), ('<phrase>background check</phrase> and <tgt> .', 34), ('There was no <tgt>', 32), ('they do nt <tgt>', 31), ('<phrase>background check</phrase> and <tgt>', 27), ('to do a <tgt>', 22), ('There is no <tgt>', 22), ('They do nt <tgt>', 22), ('to pass a <tgt> .', 21), ('They do <tgt> .', 20), ('never failed a <tgt> .', 20), ('There was no <tgt> .', 19), ('not have a <tgt> .', 19), ("do n't do <tgt>", 19), ('What kind of <tgt> do they do', 16)]


In [109]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "to take a {2}.",
        "Yes they do {2}.",
        "to pass a {2}.", 
        "they run a {2} before hiring."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'background_screening', probe_prompts, 'background_screening')
print('with relation prompts')
probe_and_eval(lm_probe, 'background_screening', probe_prompts2, 'background_screening')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'background_screening', probe_prompts, 'background_screening')
print('with relation prompts')
probe_and_eval(d_lm_probe,'background_screening', probe_prompts2, 'background_screening')

BERT-base
background_screening
     precision    recall  min_score
1     1.000000  0.012346   0.092631
2     1.000000  0.024691   0.046511
5     1.000000  0.061728   0.025006
10    0.800000  0.098765   0.016128
25    0.640000  0.197531   0.007512
50    0.640000  0.395062   0.004474
75    0.586667  0.543210   0.003032
100   0.570000  0.703704   0.002000
with relation prompts
background_screening
     precision    recall  min_score
1     1.000000  0.012346   0.039700
2     1.000000  0.024691   0.019934
5     1.000000  0.061728   0.012520
10    0.800000  0.098765   0.008386
25    0.680000  0.209877   0.003598
50    0.640000  0.395062   0.002311
75    0.586667  0.543210   0.001503
100   0.580000  0.716049   0.000915

Domain-adapted BERT
background_screening
     precision    recall  min_score
1     0.000000  0.000000  -1.000000
2     0.000000  0.000000  -1.000000
5     0.600000  0.037037   0.081086
10    0.700000  0.086420   0.021252
25    0.760000  0.234568   0.008927
50    0.620000  0.38

In [89]:
get_context(["felons", "criminals", "disabled", "drug addicts", "high schoolers", "misdemeanor", "students", "seniors"], corpus)

[('you have a <tgt> .', 8), ('I had a <tgt> ', 8), ('you have a <tgt>', 8), ('depends on the <tgt>', 8), ('not have a <tgt>', 6), ('I had a <tgt>', 6), ('on what the <tgt> was for .', 6), ('depends on the <tgt> .', 4), ('I have a <tgt> on my background', 4), ('people with a <tgt> on their record', 4), ('anyone with a <tgt> .', 4), ('someone with a <tgt> .', 4), ('you with a <tgt> ', 4), ('you have a <tgt> of <phrase>theft</phrase> but', 4), ('I have a <tgt> .', 4), ('you with a <tgt>', 4), ('person with a <tgt> ', 4), ('depending on the <tgt>', 4), ('I have a <tgt> for <phrase>assault</phrase> and', 4), ('hired with a <tgt> .', 4)]


In [110]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "hire {0}, such as {1}, {2} and {3}.",
    "people with a {2} on their record."
    ]

print('BERT-base')
probe_and_eval(lm_probe, 'person', probe_prompts, 'person')
print('with relation prompts')
probe_and_eval(lm_probe, 'person', probe_prompts2, 'person')

print()
print('Domain-adapted BERT')
probe_and_eval(d_lm_probe,'person', probe_prompts, 'person')
print('with relation prompts')
probe_and_eval(d_lm_probe,'person', probe_prompts2, 'person')

BERT-base
person
     precision    recall  min_score
1     1.000000  0.029412   0.027419
2     1.000000  0.058824   0.015791
5     0.800000  0.117647   0.005052
10    0.900000  0.264706   0.002278
25    0.640000  0.470588   0.001025
50    0.440000  0.647059   0.000608
75    0.346667  0.764706   0.000365
100   0.280000  0.823529   0.000266
with relation prompts
person
     precision    recall  min_score
1     1.000000  0.029412   0.018603
2     1.000000  0.058824   0.011497
5     0.800000  0.117647   0.003313
10    0.800000  0.235294   0.001663
25    0.600000  0.441176   0.000873
50    0.420000  0.617647   0.000513
75    0.333333  0.735294   0.000266
100   0.270000  0.794118   0.000237

Domain-adapted BERT
person
     precision    recall  min_score
1         1.00  0.029412   0.036885
2         1.00  0.058824   0.019011
5         1.00  0.147059   0.012808
10        0.60  0.176471   0.011755
25        0.68  0.500000   0.002846
50        0.48  0.705882   0.001528
75        0.36  0.794118  

## Different scoring schemes

In [114]:
def probe_and_eval_grps(lm_probe, concept, probe_prompts, prompt_groups, concept_phrase=None):
    seeds = seed_instances_dict[concept]
    if concept_phrase is None:
        cc_phrase = ' '.join(concept.split('_'))
    else:
        cc_phrase = concept_phrase
    cand_scores_per_template = []
    cand_scores = []
    
    for template in probe_prompts:
        _input_txt = template.format(cc_phrase, ', '.join(seeds[:-1]), '[MASK]', seeds[-1])
        _cand_scores = lm_probe.score_candidates(_input_txt, all_entities)
        _cand_scores.sort(key=lambda d : d["cand"])
        cand_scores_per_template.append(_cand_scores)

    for _cand_score_lst in zip(*cand_scores_per_template):
                    # _cand_score_lst: List[Dict["cand", "score"]], for the same "cand" and different template 
            _cand = _cand_score_lst[0]["cand"]
            grp_scores = {}
            for i, g in enumerate(prompt_groups):
                _score_i = _cand_score_lst[i]['score']
                grp_score_lst = grp_scores.get(g, [])
                grp_score_lst.append(_score_i)
                grp_scores[g] = grp_score_lst
            _score = 1.0
            for val in grp_scores.values():
                _score = _score * (sum(val) / len(val))
            cand_scores.append({"cand": _cand, "score": _score})

    extraction_results = []
    for d in cand_scores:
        e = d["cand"]
        if e in seeds:
            continue
        lm_score = d["score"]
        extraction_results.append({'concept': concept,
                                           'neighbor': e,
                                           'lm_score': lm_score
                                          })

        extraction_results.sort(key=lambda d : d['lm_score'], reverse=True)
    pratk(pd.DataFrame(extraction_results))

In [115]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
prompt_grps = [1,1,1]

probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "to work at {2}.",
        "{2} does not drug.",
        "while working at {2}."
    ]
prompt_grps2 = [1,1,1,2,3,2]

print('BERT-base')
probe_and_eval_grps(lm_probe, 'company', probe_prompts, prompt_grps, 'company')
print('with relation prompts')
probe_and_eval_grps(lm_probe, 'company', probe_prompts2, prompt_grps2, 'company')

print()
print('Domain-adapted BERT')
probe_and_eval_grps(d_lm_probe,'company', probe_prompts, prompt_grps, 'company')
print('with relation prompts')
probe_and_eval_grps(d_lm_probe,'company', probe_prompts2, prompt_grps2,'company')

BERT-base
company
     precision    recall  min_score
1     1.000000  0.008772   0.068087
2     1.000000  0.017544   0.046398
5     1.000000  0.043860   0.020752
10    1.000000  0.087719   0.010284
25    1.000000  0.219298   0.002511
50    0.840000  0.368421   0.000986
75    0.706667  0.464912   0.000690
100   0.600000  0.526316   0.000380
with relation prompts
company
     precision    recall     min_score
1         1.00  0.008772  6.400130e-09
2         1.00  0.017544  9.283904e-10
5         0.80  0.035088  7.966789e-10
10        0.60  0.052632  4.044021e-10
25        0.56  0.122807  8.737789e-11
50        0.58  0.254386  1.661150e-11
75        0.60  0.394737  4.061542e-12
100       0.63  0.552632  1.156959e-12

Domain-adapted BERT
company
     precision    recall  min_score
1         1.00  0.008772   0.063044
2         1.00  0.017544   0.034719
5         1.00  0.043860   0.014492
10        1.00  0.087719   0.009953
25        1.00  0.219298   0.005252
50        0.88  0.385965   0.001

In [117]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
prompt_grps = [1,1,1]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "to take a {2}.",
        "Yes they do {2}.",
        "to pass a {2}.", 
        "they run a {2} before hiring."
    ]
prompt_grps2 = [1,1,1,2,3,2,3]

print('BERT-base')
probe_and_eval_grps(lm_probe, 'background_screening', probe_prompts, prompt_grps, 'background_screening')
print('with relation prompts')
probe_and_eval_grps(lm_probe, 'background_screening', probe_prompts2, prompt_grps2, 'background_screening')

print()
print('Domain-adapted BERT')
probe_and_eval_grps(d_lm_probe,'background_screening', probe_prompts, prompt_grps, 'background_screening')
print('with relation prompts')
probe_and_eval_grps(d_lm_probe,'background_screening', probe_prompts2, prompt_grps2, 'background_screening')

BERT-base
background_screening
     precision    recall  min_score
1     1.000000  0.012346   0.092631
2     1.000000  0.024691   0.046511
5     1.000000  0.061728   0.025006
10    0.800000  0.098765   0.016128
25    0.640000  0.197531   0.007512
50    0.640000  0.395062   0.004474
75    0.586667  0.543210   0.003032
100   0.570000  0.703704   0.002000
with relation prompts
background_screening
     precision    recall     min_score
1     1.000000  0.012346  7.751327e-07
2     1.000000  0.024691  3.283780e-08
5     0.800000  0.049383  1.165075e-08
10    0.600000  0.074074  3.171605e-09
25    0.640000  0.197531  5.394475e-10
50    0.580000  0.358025  6.611104e-11
75    0.586667  0.543210  8.228445e-12
100   0.520000  0.641975  1.107889e-12

Domain-adapted BERT
background_screening
     precision    recall  min_score
1     0.000000  0.000000  -1.000000
2     0.000000  0.000000  -1.000000
5     0.600000  0.037037   0.081086
10    0.700000  0.086420   0.021252
25    0.760000  0.234568   0.

In [118]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
prompt_grps = [1,1,1]
probe_prompts2 = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
        "hire {0}, such as {1}, {2} and {3}.",
    "people with a {2} on their record."
    ]
prompt_grps2 = [1,1,1,2,2]

print('BERT-base')
probe_and_eval_grps(lm_probe, 'person', probe_prompts, prompt_grps, 'person')
print('with relation prompts')
probe_and_eval_grps(lm_probe, 'person', probe_prompts2, prompt_grps2, 'person')

print()
print('Domain-adapted BERT')
probe_and_eval_grps(d_lm_probe,'person', probe_prompts, prompt_grps, 'person')
print('with relation prompts')
probe_and_eval_grps(d_lm_probe,'person', probe_prompts2, prompt_grps2, 'person')

BERT-base
person
     precision    recall  min_score
1     1.000000  0.029412   0.027419
2     1.000000  0.058824   0.015791
5     0.800000  0.117647   0.005052
10    0.900000  0.264706   0.002278
25    0.640000  0.470588   0.001025
50    0.440000  0.647059   0.000608
75    0.346667  0.764706   0.000365
100   0.280000  0.823529   0.000266
with relation prompts
person
     precision    recall     min_score
1         1.00  0.029412  1.474955e-04
2         1.00  0.058824  7.982930e-05
5         0.80  0.117647  4.348215e-06
10        0.70  0.205882  2.206297e-06
25        0.56  0.411765  4.635252e-07
50        0.38  0.558824  1.242914e-07
75        0.32  0.705882  4.555480e-08
100       0.28  0.823529  1.098820e-08

Domain-adapted BERT
person
     precision    recall  min_score
1         1.00  0.029412   0.036885
2         1.00  0.058824   0.019011
5         1.00  0.147059   0.012808
10        0.60  0.176471   0.011755
25        0.68  0.500000   0.002846
50        0.48  0.705882   0.001528

# Relation templates

In [120]:
from roberta_ses.interface import Roberta_SES_Entailment
yutong_base_dir="/home/ubuntu/users/yutong"
roberta_ses_dir = os.path.join(yutong_base_dir, "repos", "Roberta_SES")
# 0 = contra, 1 = neutral, 2 = entail
entailment_model = Roberta_SES_Entailment(roberta_path='/home/ubuntu/users/yutong/models/roberta-large',
        ckpt_path=os.path.join(roberta_ses_dir, 'checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt'),
        max_length=512,
        device_name='cpu')

In [126]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
d_entailment_model = SentenceTransformer('/home/ubuntu/users/nikita/models/bert_finetuned_lm_sts/indeed_all/')

In [132]:
def predict_rel(sent, rel):
    return entailment_model.predict(sent, rel)[1].tolist()

def d_predict_rel(sent, rel):
    embedding1 = d_entailment_model.encode(sent, convert_to_tensor=True)
    embedding2 = d_entailment_model.encode(rel, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    return cosine_scores.item()

In [134]:
s1 = 'walmart : we have to wear uniform'
s2 = 'walmart requires uniform'
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.0018634856678545475, 0.008392865769565105, 0.9897436499595642]
0.9119620323181152


In [135]:
s1 = "kroger stores : our kroger did n't have a pharmacy when i was there , but they paid for the permit test to sell alcohol ."
s2 = "permit is needed for pharmacy ."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.003196667181327939, 0.039147090166807175, 0.9576562643051147]
0.4612185060977936


In [137]:
s1 = "cvs health : general orientation is usually about six hours ; after , pharmacy techs will be brought to the pharmacy for a separate orientation ."
s2 = "There is orientation for pharmacy ."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.002546712290495634, 0.016983652487397194, 0.9804696440696716]
0.6139006614685059


In [138]:
s1 = "walmart : 401k , health insurance , full time status as soon as there was a space open in your area ."
s2 = "walmart offers health insurance."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.0007973050232976675, 0.001054643071256578, 0.9981480836868286]
0.5119340419769287


In [139]:
s1 = "walmart : 30 - 40hrs weekly , flexible schedule ."
s2 = "walmart provides flexible schedule."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.0008081113919615746, 0.009821068495512009, 0.9893708229064941]
0.5245642066001892


In [140]:
s1 = "subway : it was a flexible schedule to go to school either morning or evenings ."
s2 = "subway provides flexible schedule."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.0009289175504818559, 0.023315617814660072, 0.975755512714386]
0.6597525477409363


In [141]:
s1 = "cvs health : after completing their training you get a raise switching you from pharmacy associate to pharmacy tech ."
s2 = "You can work training as pharmacy ."
print(predict_rel(s1,s2))
print(d_predict_rel(s1,s2))

[0.005240839906036854, 0.06974846869707108, 0.9250107407569885]
0.733309805393219


In [144]:
probe_prompts = [
        "{0}, such as {1}, {2} and {3}.",
        "{0}, including {1}, {2} and {3}.",
        "{1}, {2}, {3} and other {0}.",
    ]
concepts_to_expand = concepts
expanded_instances = {}
for c in concepts_to_expand:
    print(c)
    results = probe_and_eval(d_lm_probe,c, probe_prompts, c, evaluate=False)
    expanded_instances[c] = results

company
dress_code
job_position
pay_schedule
benefits
compensation
payment_option
background_screening
person
hire_prerequisite
shifts
schedule
employee_type
onboarding_steps


In [145]:
expanded_instances['company']

,concept,neighbor,lm_score
0,company,starbucks,6.304385e-02
1,company,nike,3.471876e-02
2,company,amazons,2.517403e-02
3,company,apple,1.895988e-02
4,company,etcs,1.808982e-02
...,...,...,...
8032,company,romans,7.788047e-09
8033,company,divorce,7.192135e-09
8034,company,jurisdiction,6.065130e-09
8035,company,gdansk,3.015721e-09


In [160]:
import re
s = "<phrase>nearly impossible</phrase> , unless you have an in with <phrase>upper</phrase> level <phrase>management</phrase> your better off finding a better job anywhere else"
phrases = re.findall(r'<phrase>(.*?)</phrase>', s)
pairs = []
if len(phrases) > 1:
    for i in range(0, len(phrases)):
        for j in range(i, len(phrases)):
            if i != j:
                pairs.append((phrases[i], phrases[j]))
                
print(pairs)

[('nearly impossible', 'upper'), ('nearly impossible', 'management'), ('upper', 'management')]


In [190]:
def get_all_mentions(corpus):
    mapping = {}
    for sent in corpus:
        phrases = re.findall(r'<phrase>(.*?)</phrase>', sent)
        pairs = []
        if len(phrases) > 1:
            for i in range(0, len(phrases)):
                for j in range(i, len(phrases)):
                    if i != j:
                        w1 = phrases[i]
                        w2 = phrases[j]
                        if w1 == w2:
                            continue
                        pair_key = str(set([w1, w2]))
                        sents = mapping.get(pair_key, [])
                        sents.append(sent)
                        mapping[pair_key] = sents
    return mapping

In [231]:
import operator
def find_context(concept1, concept2, mapping, instances):
    concept1_instances = instances[concept1]
    concept2_instances = instances[concept2]
    patterns = {}
    for c1 in concept1_instances:
        for c2 in concept2_instances:
            pair_key = str(set([c1,c2]))
#             print(pair_key)
            sents = mapping.get(pair_key, [])
            for sent in sents:
                sent = sent.replace("<phrase>", "")
                sent = sent.replace("</phrase>", "")
                i1 = sent.index(c1)
                i2 = sent.index(c2)
                if i1 == -1 or i2 == -1: 
                    continue
                start = min(i1, i2)
                end = max(i1, i2)
                offset = len(c2) if end == i2 else len(c1)
                pat = sent[start: end+offset]
#                 print(sent)
#                 print(pair_key)
#                 print(pat)
                pat = pat.replace(c1, '<src>')
                pat = pat.replace(c2, '<tgt>')
                
                left_context_words = sent[:start].split(' ')
                left_context_words = left_context_words[-min(len(left_context_words), 2):]

                right_context_words = sent[end+offset:].split(' ')
                right_context_words = right_context_words[:min(len(right_context_words), 2)]
            
                context_pat = ' '.join(left_context_words) + pat + ' '.join(right_context_words)
#                 print(context_pat)
                pat_ct = patterns.get(context_pat, 0)
                pat_ct += 1
                patterns[context_pat] = pat_ct
    return sorted(patterns.items(), key=operator.itemgetter(1),reverse=True)

In [204]:
sent_mapping = get_all_mentions(corpus)

reliable_expanded_instances = {}
for key, val in expanded_instances.items():
    sub_df = val.iloc[:min(20, len(val))]
    reliable_expanded_instances[key] = sub_df['neighbor'].tolist() + seed_instances_dict.get(key, [])

{'company': ['starbucks', 'nike', 'amazons', 'apple', 'etcs', 'macs', 'fasts', 'mcdonalds', 'wal mart', 'wal mart', 'google', 'facebook', 'qs', 'wal marts', "mcdonald ' s", 'css', 'apple store', 'kroger', "macy ' s", 'sos', 'walmart', 'amazon', 'subway', 'microsoft', 'target'], 'dress_code': ['etcs', 'facial piercings', 'nose piercings', 'hair', 'ear piercings', 'visible piercings', 'face piercings', 'body piercings', 'facial jewelry', 'colored hair', 'facial piercing', 'nose piercing', 'color hair', 'scrubs', 'leggings', 'blue scrubs', 'fragrances', 'nose rings', 'gis', 'dyes', 'business casual', 'uniform', 'hair color', 'tattoos', 'facial hair', 'shoes', 'piercings'], 'job_position': ['prep cook', 'line cook', 'customer service', 'dish washer', 'bartender', 'assistant manager', 'team member', 'night stocker', 'customer host', 'chef', 'floor associate', 'morning stocker', 'stock', 'shift leader', 'cash handling', 'manager', 'phone operator', 'food handlers', 'general manager', 'assist

In [232]:
find_context('company', 'job_position', sent_mapping, reliable_expanded_instances)

[("candidates <tgt> ca n't be trusted to make that decision ) but the panels have completely devolved into a murder trial that are the most painful and bureaucratic thing done at <src> amazon",
  14),
 ('a <tgt> or leader of the warehouse i am sure Fulltime employement at <src> amazon',
  8),
 ('A <tgt> at <src> amazon', 8),
 ('successful <tgt> at <src> amazon', 8),
 ('a <tgt> at <src> amazon', 8),
 ('the <tgt> who hired me started <src> as', 6),
 ('at <src> I was interviewed by both a <tgt> and', 6),
 ('<src> has many job openings and options from door greeter , to a cart pusher , receiving associate to a supervisor , a department <tgt> to',
  6),
 ('The <src> <src> warehouse does offer flexible hours but <src> flex is <tgt> position',
  6),
 (', <src> offers a great program to relocate to another <src> , so long as they are hiring , transfers are put above standard applications to ensure that a <tgt> gets',
  5),
 ('<src> <tgt>', 4),
 ('all <src> <src> <tgt> employees', 4),
 ('or <sr

In [234]:
find_context('job_position', 'background_screening', sent_mapping, reliable_expanded_instances)

[('hire <tgt> tested , but on a monthly basis , Corporate would randomly choose anywhere from five to twenty associate names that a <src> in',
  6),
 ('Usually <tgt> testing is random ; However , if a <src> suspects', 6),
 (', <tgt> tests are always possible , and a <src> can', 6),
 (', <tgt> are always possible , and a <src> can', 6),
 ("hire <tgt> tested , but on a monthly basis , Corporate would randomly choose anywhere from five to twenty associate names that a manager in charge had to immediately call into the <src> 's",
  6),
 ('in <src> call center we are not involve with the human resources group , which is a totally different department , and to my knowledge they do a complete <tgt> for',
  4),
 ('a <src> position they did not <tgt> test', 4),
 ('to <src> , and never once seen anyone get <tgt> drug', 4),
 ('a <src> and I can tell you that <tgt> drug', 4),
 ('ex <src> and asked to come in and complete a physical and <tgt> .', 3),
 ('the <tgt> ( they have a toll free number for 

In [235]:
find_context('job_position', 'hire_prerequisite', sent_mapping, reliable_expanded_instances)

[('good <src> and seeing how the store is free from <tgt> safety', 4),
 ('great <src> but often has to use her vacation hours to hit <tgt> labor', 4),
 ('shift <src> , they start as a shift <src> with no <tgt> to', 4),
 ('your <tgt> to your <src> and', 4),
 ('on <src> , <tgt> ,', 2),
 ('on <src> and food handling and <tgt> tips', 2),
 (', <src> and <tgt> for', 2),
 ('of <src> especially in food preparation or sales , I completed an ekg tech program , I had <tgt> in',
  2),
 ('More <tgt> intense then <src>', 2),
 ('<src> skills , computer skills , <tgt> ,', 2),
 ("'s <src> , and they will help you reset your passwords and <tgt> answers",
  2),
 ('off <tgt> , have 4 year <src> experience', 2),
 ('a <tgt> to become an <src>', 2),
 ('an <src> application was as little as half a year to a year of <tgt> ,', 2),
 ('<tgt> <src> .', 2),
 ('a <tgt> vest while spotting another associate that would be on a <src> picker',
  2),
 ("did <tgt> , q'ing ovens , grill , fryers , <src> ,", 2),
 ('of <src>